# Week 26 Hacks 

- title: Week 26 hacks 
- toc: true
- author: Khalid Farah
- categories: [collegeboard]
- tags: [python]

# Create my own 2.4 A

In [54]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of databaseapp.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)

In [55]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''
class Cars(db.Model):
    __tablename__ = 'cars'  # table name is plural, class name is singular

    id = db.Column(db.Integer, primary_key=True)
    _company = db.Column(db.String(255), unique=False, nullable=False) #_name
    _model = db.Column(db.String(255), unique=False, nullable=False) #_uid
    _price = db.Column(db.String(255), unique=False, nullable=False) # _password

    def __init__(self, company, model, price):
        self._company = company    # variables with self prefix become part of the object, 
        self._model = model
        self._price = price

    @property
    def company(self):
        return self._company
    
    @company.setter
    def company(self, company):
        self._company = company
    
    @property
    def model(self):
        return self._model
    
    @model.setter
    def model(self, model):
        self._model = model
        
    def is_model(self, model):
        return self._model == model
    
    @property
    def price(self):
        return self._price
    
    @price.setter
    def price(self, price):
        self._price = price
    
    def __str__(self):
        return json.dumps(self.read())

    def create(self):
        try:
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    def read(self):
        return {
            "id": self.id,
            "company": self.company,
            "model": self.model,
            "price": self.price,
        }

    def update(self, company="", model="", price=""):
        """only updates values with length"""
        if len(company) > 0:
            self.company = company
        if len(model) > 0:
            self.model = model
        if len(price) > 0:
            self.price = price
        db.session.commit()
        db.session.add(self)
        return self

    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None

In [57]:
"""Database Creation and Testing """


# Builds working data for testing
def initCars():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        c1 = Cars(company='Bmw', model='3 Series', price='20,000')
        c2 = Cars(company='Bmw', model='4 Series', price='15,000')
        c3 = Cars(company='Toyota', model='Corolla', price='10,000')
        c4 = Cars(company='Toyota', model='Camry', price='30,000')
        c5 = Cars(company='Toyota', model='Prius', price='25,000')
        c6 = Cars(company='Honda', model='Accord', price='50,000')

        cars = [c1, c2, c3, c4, c5, c6]

        """Builds sample user/note(s) data"""
        for cars in cars:
            try:
                '''add user to table'''
                object = cars.create()
                print(f"Created new uid {object.model}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {cars.model}, or error.")
                
initCars()



Created new uid 3 Series
Created new uid 4 Series
Created new uid Corolla
Created new uid Camry
Created new uid Prius
Created new uid Accord


In [49]:
def find_by_model(model):
    with app.app_context():
        cars = Cars.query.filter_by(_model=model).first()
    return cars

Create 

In [50]:
def create():
    # optimize user time to see if uid exists
    model = input("Enter your model:")
    cars = find_by_model(model)
    try:
        print("Found\n", Cars.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    company = input("Enter the company:")
    price = input("Enter the price")
    
    # Initialize User object before date
    cars = Cars(company=company, 
                model=model, 
                price=price,
                )
    
    # create user.dob, fail with today as dob
    # dob = input("Enter your date of birth 'YYYY-MM-DD'")
    # try:
    #     user.dob = datetime.strptime(dob, '%Y-%m-%d').date()
    # except ValueError:
    #     user.dob = datetime.today()
    #     print(f"Invalid date {dob} require YYYY-mm-dd, date defaulted to {user.dbo}")
           
    # write object to database
    with app.app_context():
        try:
            object = cars.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
create()

Created
 {'id': 7, 'company': 'BMW', 'model': '7 series', 'price': '50,000'}


Read

In [52]:
def read():
    with app.app_context():
        table = Cars.query.all()
    json_ready = [cars.read() for cars in table] # each user adds user.read() to list
    return json_ready

read()

[{'id': 1, 'company': 'Bmw', 'model': '3 Series', 'price': '20,000'},
 {'id': 2, 'company': 'Bmw', 'model': '4 Series', 'price': '15,000'},
 {'id': 3, 'company': 'Toyota', 'model': 'Corolla', 'price': '10,000'},
 {'id': 4, 'company': 'Toyota', 'model': 'Camry', 'price': '30,000'},
 {'id': 5, 'company': 'Toyota', 'model': 'Prius', 'price': '25,000'},
 {'id': 6, 'company': 'Honda', 'model': 'Accord', 'price': '50,000'},
 {'id': 7, 'company': 'BMW', 'model': '7 series', 'price': '50,000'}]

Delete

In [58]:
def delete_by_company(): # makes a new function called delete_by_uid
    model = input("Enter uid of user to be deleted.") # prompts the user to enter the uid
    user = find_by_model(model)  # using previous function to locate user by inputted id
    with app.app_context():
        try:
            object = user.delete() 
            print(f"User with uid --{model}-- has been deleted. ")
            db = read()
            print(db)
        except:  # error raised if object not found
           (f"No user with uid {model} was found.")
        
delete_by_company()

Understanding Debugging 
![]({{site.baseurl}}/images/Screenshot 2023-03-21 201138.png)


2.4 B hacks 

Do I see procedural abstraction?
yes, in this implementation, i do see a procedural abstraction. using procedural abstraction, the code can write code that is easier to read and maintain.

- Recalling the College Board criteria, data abstraction is essentially representing something (i.e a program) in a much more simplified, condensed method while hiding its more internal functions
- Looking at this implementation, I do see data abstraction in this implementation. The implementation hides the details of the underlying database operations from the user, providing a simple interface for performing basic CRUD (Create, Read, Update, Delete) operations
- Regarding debugging, one possible example is if the user enters invalid input (e.g., a non-existent uid when updating or deleting a user). An example would be the implementation catches the exception raised by the database operation and prints an error message to the user
- The error message provides a high-level explanation of the problem ("No uid was found in the table"), without revealing the underlying implementation details. This is an example of data abstraction, as the implementation is hiding the details of the database operation (in this case, the SQL query) from the user and providing a simplified, high-level interface for working with the data.


![]({{site.baseurl}}/images/Screenshot 2023-03-21 201436.png)
